In [47]:
# import all the things!
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import os
from sklearn.linear_model import Ridge
from sklearn.grid_search import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.cross_validation import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import roc_auc_score

In [2]:
%pwd

u'/Users/Driz/DSI-CHI-1'

In [8]:
data = pd.read_csv('../titanic.csv')

In [9]:
data.head()

,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [48]:
# overview of whats going on in my data
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
index          891 non-null int64
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(6), object(5)
memory usage: 90.6+ KB


In [10]:
data.columns

Index([u'index', u'PassengerId', u'Survived', u'Pclass', u'Name', u'Sex',
       u'Age', u'SibSp', u'Parch', u'Ticket', u'Fare', u'Cabin', u'Embarked'],
      dtype='object')

In [11]:
data.describe()

/Users/Driz/anaconda2/lib/python2.7/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,index,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,445.000000,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,222.500000,223.500000,0.000000,2.000000,NaN,0.000000,0.000000,7.910400
50%,445.000000,446.000000,0.000000,3.000000,NaN,0.000000,0.000000,14.454200
75%,667.500000,668.500000,1.000000,3.000000,NaN,1.000000,0.000000,31.000000
max,890.000000,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [12]:
data.isnull().any()

index          False
PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool

In [30]:
# age, cabin, and embarked have missing data
# explore further...

In [10]:
VARIABLE DESCRIPTIONS:
survival        Survival
                (0 = No; 1 = Yes)
pclass          Passenger Class
                (1 = 1st; 2 = 2nd; 3 = 3rd)
name            Name
sex             Sex
age             Age
sibsp           Number of Siblings/Spouses Aboard
parch           Number of Parents/Children Aboard
ticket          Ticket Number
fare            Passenger Fare
cabin           Cabin
embarked        Port of Embarkation
                (C = Cherbourg; Q = Queenstown; S = Southampton)

SPECIAL NOTES:
Pclass is a proxy for socio-economic status (SES)
 1st ~ Upper; 2nd ~ Middle; 3rd ~ Lower

Age is in Years; Fractional if Age less than One (1)
 If the Age is Estimated, it is in the form xx.5

With respect to the family relation variables (i.e. sibsp and parch)
some relations were ignored.  The following are the definitions used
for sibsp and parch.

Sibling:  Brother, Sister, Stepbrother, or Stepsister of Passenger Aboard Titanic
Spouse:   Husband or Wife of Passenger Aboard Titanic (Mistresses and Fiances Ignored)
Parent:   Mother or Father of Passenger Aboard Titanic
Child:    Son, Daughter, Stepson, or Stepdaughter of Passenger Aboard Titanic

Other family relatives excluded from this study include cousins,
nephews/nieces, aunts/uncles, and in-laws.  Some children travelled
only with a nanny, therefore parch=0 for them.  As well, some
travelled with very close friends or neighbors in a village, however,
the definitions do not support such relations.

SyntaxError: invalid syntax (<ipython-input-10-f1cc15b5c1fb>, line 1)

In [35]:
titanic_data = pd.DataFrame(data)

In [14]:
titanic_data.head()

,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [52]:
# practice loops, obviously need more practice
dead_or_alive = []
for 'Name' in titanic_data:
    if 'Survived' == 1
    return "s% is alive and well" %
print dead_or_alive

SyntaxError: invalid syntax (<ipython-input-52-648d99d78eaa>, line 3)

In [15]:
titanic_data['Embarked'].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

In [31]:
# thats not too bad
# might just drop those rows
titanic_data['Embarked'].isnull().value_counts()

False    889
True       2
Name: Embarked, dtype: int64

In [16]:
titanic_data['Survived'].describe()

count    891.000000
mean       0.383838
std        0.486592
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Survived, dtype: float64

In [17]:
titanic_data['Survived'].sum()

342

In [47]:
# here I want to loop through all the names of people on board and print out whether or not they survived

alive_or_dead = []
for i in range (len(titanic_data['Survived'])):
    for i in (titanic_data['Survived']):
        for j in i
    print "s% did not survive"

SyntaxError: invalid syntax (<ipython-input-47-a13606907a11>, line 3)

In [58]:
alive_or_dead = []
for i in range (len(titanic_data['Survived'])):
    for i in (titanic_data['Survived']) if i == 0:
        for j in i(titanic_data['Name']):
        print "s% did not survive" %

SyntaxError: invalid syntax (<ipython-input-58-0f460f526e6b>, line 3)

In [18]:
titanic_data.head()

,index,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [19]:
titanic_data['Cabin'].isnull().any()

True

In [20]:
titanic_data['Cabin'].isnull().value_counts()

True     687
False    204
Name: Cabin, dtype: int64

In [21]:
titanic_data['Name'].isnull().value_counts()

False    891
Name: Name, dtype: int64

In [22]:
titanic_data['Survived'].isnull().value_counts()

False    891
Name: Survived, dtype: int64

In [23]:
# think about what I want to do with the missing values
# possibly fillna with average or dropna.....think think
titanic_data['Age'].isnull().value_counts()

False    714
True     177
Name: Age, dtype: int64

In [24]:
# will want to use two of these variables--multicolinearity
titanic_data['Pclass'].value_counts()

3    491
1    216
2    184
Name: Pclass, dtype: int64

In [46]:
# wow, thats quite a range in fare prices
# what could I do with this...
# could make various fare bins
# also, why did 15 people pay 0.00 ???
titanic_data['Fare'].value_counts(ascending=True)

81.8583      1
15.1000      1
76.2917      1
10.1708      1
61.1750      1
8.8500       1
12.6500      1
221.7792     1
13.7917      1
22.0250      1
6.8583       1
63.3583      1
8.5167       1
15.5500      1
6.4500       1
22.5250      1
12.2875      1
12.2750      1
9.8417       1
15.0000      1
14.0000      1
7.7875       1
8.4583       1
12.0000      1
4.0125       1
15.0458      1
7.1417       1
21.6792      1
15.0500      1
6.9500       1
            ..
26.2500      6
30.0000      6
7.7958       6
27.9000      6
14.4542      7
31.2750      7
69.5500      7
52.0000      7
56.4958      7
7.0500       7
14.5000      7
24.1500      8
15.5000      8
16.1000      9
9.5000       9
7.2250      12
7.2500      13
7.8542      13
8.6625      13
0.0000      15
26.5500     15
7.2292      15
7.7750      16
7.9250      18
10.5000     24
26.0000     31
7.7500      34
7.8958      38
13.0000     42
8.0500      43
Name: Fare, dtype: int64

In [29]:
titanic_data['Fare'].dtypes

dtype('float64')

In [32]:
titanic_data['Fare'].std()

49.693428597180905

In [1]:
titanic_data(['Fare'],ascending=True)

NameError: name 'titanic_data' is not defined

In [33]:
titanic_data['Fare'].mean()

32.2042079685746

In [39]:
# looks as though some fares are listed at 0.00
# need to further investigate this...
titanic_data['Fare'].describe()

count    891.000000
mean      32.204208
std       49.693429
min        0.000000
25%        7.910400
50%       14.454200
75%       31.000000
max      512.329200
Name: Fare, dtype: float64

My inclination is that the fares == 0.00 are complimentary tickets. Need to make a table showing all info on people whose fare was 0.00 and then dig a little deeper to confirm.

In [34]:
titanic_data['Age'].mean()

29.69911764705882

In [38]:
titanic_data['Age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%             NaN
50%             NaN
75%             NaN
max       80.000000
Name: Age, dtype: float64

In [31]:
# can explore different age groups to see if that impacts survival rate
old_people = data[data['Age'] > 50][['Sex', 'Pclass', 'Age', 'Survived']]
old_people.head()

,Sex,Pclass,Age,Survived
6,male,1,54.0,0
11,female,1,58.0,1
15,female,2,55.0,1
33,male,2,66.0,0
54,male,1,65.0,0


In [32]:
old_people.describe()

,Pclass,Age,Survived
count,64.000000,64.000000,64.000000
mean,1.546875,58.718750,0.343750
std,0.754451,6.528858,0.478714
min,1.000000,51.000000,0.000000
25%,1.000000,54.000000,0.000000
50%,1.000000,58.000000,0.000000
75%,2.000000,62.000000,1.000000
max,3.000000,80.000000,1.000000


In [33]:
old_people['Survived'].value_counts()

0    42
1    22
Name: Survived, dtype: int64

In [ ]:
# start making some pivot tables and dummie_var for columns 
# then get to modeling!